In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
from google.colab import drive
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,DepthwiseConv2D,Input,Add,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_score, recall_score, accuracy_score,classification_report,confusion_matrix
import seaborn as sns

In [4]:
drive.mount('/content/drive')
dataset= '/content/drive/MyDrive/Plant_Disease'
print(os.listdir(dataset))
model=load_model("/content/drive/MyDrive/Plant_Disease/Apple_Disease_4classes_64_2.keras")

Mounted at /content/drive
['VGG-19.ipynb', 'Apple_valid', 'training_history.json', 'Hybrid-Vgg-19-DepthRes.ipynb', 'training_history_4096.json', 'training_history_2048.json', 'File-uploads.ipynb', 'Testing', 'training_history_512.json', 'Apple_2', 'Apple_3', 'VGG-19-Depthwise.ipynb', 'Apple', 'training_history_512_2.json', 'training_history_512_2_1.json', 'u2net_portrait.pth', 'bg-removed', 'Apple_bg_removed', 'Apple_valid_bg_removed', 'training_history_64.json', 'Apple_Testing', 'Apple_Testing_bg_removed', 'Testing_bg_removed', 'Testing_apple', 'Testing_bg_removed_2', 'Apple_Disease_4classes_64.json', 'Apple_Disease_4classes_64.keras', 'Apple_Disease_4classes_64_weights.weights.h5', 'Apple_Disease_4classes_64_2.json', 'Apple_Disease_4classes_64_2.keras', 'Apple_Disease_4classes_64_weights_2.weights.h5', 'Hybrid-Vgg-19.ipynb', 'Hybrid-vgg19-64.ipynb', 'Testing test set.ipynb']


In [5]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ depthwise_conv2d          │ (None, 224, 224, 3)    │             30 │ input_layer[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 224, 224, 64)   │            256 │ depthwise_conv2d[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 224, 224, 64)   │              0 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ depthwise_conv2d_1        │ (None, 224, 224, 64)   │            640 │ activation[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 224, 224, 64)   │              0 │ depthwise_conv2d_1[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 224, 224, 64)   │          4,160 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 224, 224, 64)   │              0 │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 112, 112, 64)   │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ depthwise_conv2d_2        │ (None, 112, 112, 64)   │            640 │ max_pooling2d[0][0]    │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 112, 112, 64)   │              0 │ depthwise_conv2d_2[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 112, 112, 128)  │          8,320 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ depthwise_conv2d_3        │ (None, 112, 112, 128)  │          1,280 │ conv2d_3[0][0]         │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activation) │ (None, 112, 112, 128)  │              0 │ depthwise_conv2d_3[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 112, 112, 128)  │         16,512 │ activation_4[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 112, 112, 128)  │          8,320 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 112, 112, 128)  │              0 │ conv2d_4[0][0],        │
│                      

 Total params: 368,606,120 (1.37 GB)

 Trainable params: 122,868,706 (468.71 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 245,737,414 (937.41 MB)

In [6]:
dir = os.path.join(dataset, 'Testing_bg_removed_2')
sets = tf.keras.utils.image_dataset_from_directory(
    dir,
    labels="inferred",
    # Change label_mode to 'categorical' for one-hot encoding
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)
class_name = sets.class_names

Found 1010 files belonging to 4 classes.


In [8]:
from collections import Counter
def count_classes(dataset, class_names):
    class_counts = Counter()
    for images, labels in dataset:
        label_indices = np.argmax(labels.numpy(), axis=1)
        for label_idx in label_indices:
            class_counts[class_names[label_idx]] += 1
    return class_counts

class_names = sets.class_names

In [9]:
test_counts = count_classes(sets,class_names)

print("Testing Set2:")
for class_name, count in test_counts.items():
    print(f"{class_name}: {count}")



Testing Set2:
Apple___healthy: 255
Apple___Black_rot: 266
Apple___Apple_scab: 281
Apple___Cedar_apple_rust: 208


In [10]:
predictions= []
true_labels = []
for images, labels in sets:
    preds = model.predict(images)
    predictions.extend(np.argmax(preds, axis=1))
    true_labels.extend(np.argmax(labels.numpy(), axis=1))

predictions = np.array(predictions)
true_labels = np.array(true_labels)

for i in range(1010):
    print(f"Image {i+1}: Predicted - {class_names[predictions[i]]}, True - {class_names[true_labels[i]]}")

accuracy = np.mean(predictions == true_labels)
print(f"\nTesting Accuracy: {accuracy:.2%}")

report = classification_report(
    true_labels,
    predictions,
    target_names=class_names,
    digits=2
)
print("\nClassification Report:\n")
print(report)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
accuracy = accuracy_score(true_labels,predictions)
print(f"Accuracy: {accuracy:.4f}")
precision = precision_score(true_labels, predictions, average='weighted')
print(f"Precision: {precision:.4f}")
recall = recall_score(true_labels,predictions, average='weighted')
print(f"Recall: {recall:.4f}")

In [ ]:
cm = confusion_matrix(true_labels, predictions, labels=np.arange(len(class_names)))


plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='magma', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()